Set up Table for use

In [88]:
import duckdb

In [89]:
con = duckdb.connect()
con.execute("DROP TABLE IF EXISTS novel_table")
con.execute("DROP TABLE IF EXISTS country_novel_table")
table = f"""
    CREATE TABLE novel_table AS
    SELECT * FROM 'webnovel_2025_analysis.csv'
"""
con.execute(table)

country_table = con.execute("CREATE TABLE country_novel_table AS SELECT * FROM 'webnovel_2025_analysis.csv' WHERE CountryOfOrigin == 'KR' OR CountryOfOrigin == 'CN' OR CountryOfOrigin == 'JP'").fetchdf()

query = "ALTER TABLE country_novel_table ADD COLUMN TotalStars DECIMAL(10, 2)"
con.execute(query)

query = "UPDATE country_novel_table SET TotalStars = NumberOfReviews * StarRating"
con.execute(query)

query = "UPDATE country_novel_table SET NumberOfChapters = REPLACE(NumberOfChapters, ' Chapters', '')"
con.execute(query)

query = "UPDATE country_novel_table SET LastChapterDateOfPublication = NULL WHERE LastChapterDateOfPublication == 'N/A'"
con.execute(query)

Begin Final Analysis. The goals are to discover:
- Which country has the highest number of total stars divided by the number of novels by that country
- Find which country has the most readers
- Look into Genres

In [62]:
query = f"""SELECT CountryOfOrigin, 
            SUM(TotalStars) / SUM(NumberOfReviews) AS AvgStarsPerReviewer
            FROM country_novel_table
            GROUP BY CountryOfOrigin
            ORDER BY AvgStarsPerReviewer DESC;"""
print(con.execute(query).fetch_df())

  CountryOfOrigin  AvgStarsPerReviewer
0              CN             4.059688
1              KR             3.967862
2              JP             3.825856


This means on a purely review based system chinese novels have the highest number of reviews

In [100]:
query = f"""SELECT CountryOfOrigin, COUNT(Title) AS NumberOfTitles
            FROM country_novel_table
            GROUP BY CountryOfOrigin
            ORDER BY NumberOfTitles DESC;"""
print(con.execute(query).fetch_df())

  CountryOfOrigin  NumberOfTitles
0              CN           14418
1              JP            7485
2              KR            6800


In [66]:
query = f"""SELECT CountryOfOrigin, SUM(NumberOfReaders) AS NumberOfReaders, COUNT(Title) AS NumberOfTitles, SUM(NumberOfReaders) / COUNT(Title) AS AverageReaderPerTitle
            FROM country_novel_table
            GROUP BY CountryOfOrigin
            ORDER BY NumberOfReaders DESC;"""
print(con.execute(query).fetch_df())

  CountryOfOrigin  NumberOfReaders  NumberOfTitles  AverageReaderPerTitle
0              CN       17696720.0           14418            1227.404633
1              JP        9771135.0            7485            1305.428858
2              KR        7163718.0            6800            1053.487941


In [83]:
query = f"""SELECT CountryOfOrigin, COUNT(NumberOfChapters) AS numZeroChapterEntries
            FROM country_novel_table
            WHERE NumberOfChapters = 0
            GROUP BY CountryOfOrigin
            ORDER BY numZeroChapterEntries DESC;"""
print(con.execute(query).fetch_df())

  CountryOfOrigin  numZeroChapterEntries
0              CN                   1001
1              JP                    668
2              KR                    593


In [56]:
query = f"""
SELECT CountryOfOrigin, COUNT(*) AS NumberOfTitles
FROM country_novel_table
WHERE TRY_CAST(REPLACE(Rank, '#', '') AS INT) < 100
GROUP BY CountryOfOrigin
ORDER BY NumberOfTitles DESC
"""
print(con.execute(query).fetch_df())

  CountryOfOrigin  NumberOfTitles
0              CN              41
1              JP              36
2              KR              22


In [35]:
query = f"""SELECT trimmed_genre AS Genre, 
                COUNT(*) AS GenreCount, 
                AVG(StarRating) AS AvgRatingPerGenre,
                SUM(TotalStars) / SUM(NumberOfReviews) AS AvgStarsPerReview
            FROM (
                SELECT UNNEST(STRING_SPLIT(TRIM(BOTH '()' FROM Genres), ' ')) AS trimmed_genre, 
                    TotalStars, 
                    StarRating,
                    NumberOfReviews
                FROM country_novel_table
            )
            WHERE Genre != 'Slice' AND Genre != 'of'
            GROUP BY trimmed_genre
            ORDER BY GenreCount DESC;
            """
print(con.execute(query).fetch_df())

            Genre  GenreCount  AvgRatingPerGenre  AvgStarsPerReview
0          Sports         223           3.736771           4.272974
1              Ai        2405           3.927069           4.264541
2          Horror         949           3.741096           4.209186
3            Yaoi        4363           3.858950           4.175625
4         Mystery        2567           3.701519           4.163922
5      Historical        1771           3.628458           4.161898
6   Psychological        2356           3.728905           4.150410
7         Xianxia        1120           3.629643           4.146964
8           Wuxia         408           3.721078           4.139499
9         Shounen        3919           3.774713           4.096014
10      Completed        6876           3.755163           4.091777
11   Supernatural        4140           3.691787           4.086253
12         Comedy        9263           3.675386           4.073512
13           Life        9950           3.634472

In [39]:
query = f"""WITH GenreCounts AS (
                SELECT CountryOfOrigin, 
                    trimmed_genre AS Genre, 
                    COUNT(*) AS GenreCount,
                    SUM(TotalStars) / SUM(NumberOfReviews) AS aveReview,
                    ROW_NUMBER() OVER (PARTITION BY CountryOfOrigin ORDER BY COUNT(*) DESC) AS rank
                FROM (
                    SELECT CountryOfOrigin, TotalStars, NumberOfReviews,
                        UNNEST(STRING_SPLIT(TRIM(BOTH '()' FROM Genres), ' ')) AS trimmed_genre
                    FROM country_novel_table
                )
                GROUP BY CountryOfOrigin, trimmed_genre
            )
            SELECT CountryOfOrigin, Genre, GenreCount, aveReview
            FROM GenreCounts
            WHERE rank <= 5
            ORDER BY CountryOfOrigin, rank;
            """

print(con.execute(query).fetch_df())

   CountryOfOrigin    Genre  GenreCount  aveReview
0               CN  Romance        9497   4.127164
1               CN  Fantasy        6040   4.008719
2               CN    Drama        4811   4.093541
3               CN     Life        4591   4.135920
4               CN   Comedy        4300   4.140948
5               JP  Fantasy        4533   3.809106
6               JP  Romance        4323   3.827013
7               JP     Life        3869   3.885210
8               JP   Comedy        3179   3.858635
9               JP   Action        2684   3.773606
10              KR  Fantasy        5626   3.964622
11              KR  Romance        4502   3.942207
12              KR    Drama        2664   4.001458
13              KR   Action        2083   4.017693
14              KR   Comedy        1784   4.056200


In [26]:
query = f"""WITH GenreStats AS (
                SELECT CountryOfOrigin, 
                    trimmed_genre AS Genre, 
                    COUNT(*) AS GenreCount,
                    SUM(TotalStars) / NULLIF(SUM(NumberOfReviews), 0) AS AvgStarsPerReview,
                    ROW_NUMBER() OVER (PARTITION BY CountryOfOrigin ORDER BY COUNT(*) DESC) AS rank
                FROM (
                    SELECT CountryOfOrigin, 
                        UNNEST(STRING_SPLIT(TRIM(BOTH '()' FROM Genres), ' ')) AS trimmed_genre,
                        TotalStars,
                        NumberOfReviews
                    FROM country_novel_table
                )
                GROUP BY CountryOfOrigin, trimmed_genre
            )
            SELECT CountryOfOrigin, Genre, GenreCount, AvgStarsPerReview
            FROM GenreStats
            WHERE rank <= 5
            ORDER BY CountryOfOrigin, rank;
            """
print(con.execute(query).fetch_df())

   CountryOfOrigin    Genre  GenreCount  AvgStarsPerReview
0               CN  Romance        9497           4.127164
1               CN  Fantasy        6040           4.008719
2               CN    Drama        4811           4.093541
3               CN     Life        4591           4.135920
4               CN   Comedy        4300           4.140948
5               JP  Fantasy        4533           3.809106
6               JP  Romance        4323           3.827013
7               JP     Life        3869           3.885210
8               JP   Comedy        3179           3.858635
9               JP   Action        2684           3.773606
10              KR  Fantasy        5626           3.964622
11              KR  Romance        4502           3.942207
12              KR    Drama        2664           4.001458
13              KR   Action        2083           4.017693
14              KR   Comedy        1784           4.056200


In [99]:
query = f"""SELECT CountryOfOrigin, AVG(ReleaseFrequency) AS averageReleaseSchedule
            FROM country_novel_table
            WHERE ReleaseFrequency < 90
            GROUP BY CountryOfOrigin
            ORDER BY averageReleaseSchedule ASC;"""
print(con.execute(query).fetch_df())

  CountryOfOrigin  averageReleaseSchedule
0              KR               18.430493
1              CN               22.380114
2              JP               28.194972
